In [1]:
# import required libraries
import os
import datetime
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    CodeConfiguration,
    ProbeSettings,
    Environment,
    Model,
)
from azure.ai.ml.constants import AssetTypes

from azureml.core import (
    Experiment,
    Workspace,
    Run
)


from azure.identity import DefaultAzureCredential

In [3]:
# get experiment
name = "trainv1-with-epochs"
latest_run_id = ""

In [4]:
workspace = Workspace.from_config()
credential = DefaultAzureCredential()
ml_client=MLClient.from_config(credential)

Found the config file in: /config.json


In [28]:
model_path = f"azureml://subscriptions/{workspace.subscription_id}/resourcegroups/{workspace.resource_group}/workspaces/{workspace.name}/datastores/{workspace.get_default_datastore().name}/paths/dataset/{latest_run.id}/nnUNet/"


In [5]:
model_path=f"azureml://subscriptions/{workspace.subscription_id}/resourcegroups/{workspace.resource_group}/workspaces/{workspace.name}/datastores/{workspace.get_default_datastore().name}/paths/dataset/0148b05c-223c-4158-b7a5-e7a989a2f4cd/nnUNet"

In [6]:


exp = Experiment(workspace, name)
# get experiment's latest run id
if latest_run_id:
    latest_run = Run(exp, latest_run_id) 
else:
    latest_run = next(exp.get_runs())
print(latest_run)
print(type(latest_run))


Run(Experiment: trainv1-with-epochs,
Id: 5fb66d08-051e-4db5-b453-d1182fa6875d,
Type: azureml.PipelineRun,
Status: Completed)
<class 'azureml.core.run.Run'>


In [10]:
model_path = f"azureml://subscriptions/{workspace.subscription_id}/resourcegroups/{workspace.resource_group}/workspaces/{workspace.name}/datastores/{workspace.get_default_datastore().name}/paths/dataset/{latest_run.id}/nnUNet/"


In [95]:
model_path = f"azureml://subscriptions/{workspace.subscription_id}/resourcegroups/{workspace.resource_group}/workspaces/{workspace.name}/datastores/{workspace.get_default_datastore().name}/paths/dataset/{latest_run.id}/nnUNet/"
# model_path=f"azureml://subscriptions/dc3b2a9f-9106-4946-9d81-3f115060b468/resourcegroups/databricks-workspace-rg1/workspaces/edp-mlw-mlmm-tst-scentral-01/datastores/workspaceblobstore/paths/dataset/49e700cc-71be-45f0-9b83-bb2ff9a7e53f/nnunet_outputs/"

In [7]:
print(model_path)

azureml://subscriptions/dc3b2a9f-9106-4946-9d81-3f115060b468/resourcegroups/databricks-workspace-rg1/workspaces/edp-mlw-mlmm-tst-scentral-01/datastores/workspaceblobstore/paths/dataset/0148b05c-223c-4158-b7a5-e7a989a2f4cd/nnUNet


In [8]:
# model registration procedure
model = Model(path=model_path,
                name="nnunet_model_test",
                description="test",
                type=AssetTypes.CUSTOM_MODEL)
registered_model = ml_client.models.create_or_update(model)

In [9]:
# Creating a unique endpoint name with current datetime to avoid conflicts
online_endpoint_name = "nnunet-model-" + datetime.datetime.now().strftime(
    "%m%d%H%M%f"
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="sample online endpoint for deploying model",
    auth_mode="key",
    tags={"foo": "bar"},
)
print(online_endpoint_name)

nnunet-model-07071509974494


In [10]:

ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://nnunet-model-07071509974494.southcentralus.inference.ml.azure.com/score', 'openapi_uri': 'https://nnunet-model-07071509974494.southcentralus.inference.ml.azure.com/swagger.json', 'name': 'nnunet-model-07071509974494', 'description': 'sample online endpoint for deploying model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/dc3b2a9f-9106-4946-9d81-3f115060b468/resourcegroups/databricks-workspace-rg1/providers/microsoft.machinelearningservices/workspaces/edp-mlw-mlmm-tst-scentral-01/onlineendpoints/nnunet-model-07071509974494', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/dc3b2a9f-9106-4946-9d81-3f115060b468/providers/Microsoft.MachineLearningServices/locations/southcentralus/mfeOperationsStatus/oe:f04bc5dd-c904-4813-8223-972abc7635c1:51019d1b-afd8-4911-aed7-3f380dbf7fab?api-version=2022-02-01-preview'}, 'id': 

In [23]:
#docker_env.docker.base_image = "nnunet.azurecr.io/training_env" # example : "fastdotai/fastai2:latest"
#docker_env.python.user_managed_dependencies = True

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    environment=Environment(image="edpcrmlmmtst01.azurecr.io/prediction_env"),
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS2_v2", #"Standard_DS2_v2",
    code_configuration=CodeConfiguration(code="", scoring_script="score.py"),
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [24]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint nnunet-model-07071509974494 exists
Uploading nnunet (0.13 MBs): 100%|██████████| 127489/127489 [00:00<00:00, 1882658.13it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


.................................

HttpResponseError: (None) ResourceNotReady: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
Code: None
Message: ResourceNotReady: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready

In [56]:
# update deployment to take 100% traffic
endpoint.traffic = {name: 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://nnunet-model-05162126162498.southcentralus.inference.ml.azure.com/score', 'openapi_uri': 'https://nnunet-model-05162126162498.southcentralus.inference.ml.azure.com/swagger.json', 'name': 'nnunet-model-05162126162498', 'description': 'sample online endpoint for deploying model', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/dc3b2a9f-9106-4946-9d81-3f115060b468/resourcegroups/databricks-workspace-rg1/providers/microsoft.machinelearningservices/workspaces/edp-mlw-mlmm-tst-scentral-01/onlineendpoints/nnunet-model-05162126162498', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/dc3b2a9f-9106-4946-9d81-3f115060b468/providers/Microsoft.MachineLearningServices/locations/southcentralus/mfeOperationsStatus/oe:f04bc5dd-c904-4813-8223-972abc7635c1:66c67b64-e701-47ea-8a72-7dd20bf51c47?api-version=2022-02-01-preview'}, 'id': 